# 03 - 結構化輸出 (Structured Output)

## 📚 學習目標

- ✅ 理解為什麼需要結構化輸出
- ✅ 使用 Pydantic 定義複雜的輸出結構
- ✅ 掌握 `with_structured_output()` 方法
- ✅ 學習 Tool Strategy 與 Provider Strategy
- ✅ 實作國泰人壽保單資料結構化系統

## ⏰ 預計時間: 40 分鐘

## 📦 環境設定

In [ ]:
!pip install langchain==0.1.0
!pip install langchain-openai==0.0.5
!pip install pydantic==2.5.0
!pip install python-dotenv==1.0.0

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
print("✅ 環境設定完成")

## Part 1: 為什麼需要結構化輸出?

### 1.1 問題場景

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=0)

# 非結構化輸出
response = llm.invoke("請提供保單 CL202401001 的資訊,包含投保人、保額、保費、狀態")

print("❌ 非結構化輸出:")
print(response.content)
print(f"\n類型: {type(response.content)}")
print("\n問題: 難以程式化處理,無法直接存入資料庫或 JSON API")

### 1.2 結構化輸出的優勢

✅ **使用結構化輸出後**:
```python
{
  "policy_id": "CL202401001",
  "owner": "王小明",
  "coverage": 2000000,
  "premium": 50000,
  "status": "active"
}
```

優點:
- 🔄 可直接轉換為 JSON
- 💾 可存入資料庫
- 🔍 可進行資料驗證
- 🎯 類型安全
- 📊 易於統計分析

## Part 2: 使用 Pydantic 定義結構

### 2.1 基礎 Pydantic Model

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import date

class PolicyInfo(BaseModel):
    """保單基本資訊"""
    policy_id: str = Field(description="保單號碼")
    policy_type: str = Field(description="保單類型: 終身壽險/定期壽險/醫療險等")
    owner_name: str = Field(description="投保人姓名")
    coverage_amount: int = Field(description="保額 (新台幣)", gt=0)
    annual_premium: int = Field(description="年繳保費 (新台幣)", gt=0)
    status: str = Field(description="保單狀態: active/inactive/pending")
    start_date: str = Field(description="保單生效日期 YYYY-MM-DD")
    
# 測試建立實例
example_policy = PolicyInfo(
    policy_id="CL202401001",
    policy_type="終身壽險",
    owner_name="王小明",
    coverage_amount=2000000,
    annual_premium=50000,
    status="active",
    start_date="2024-01-15"
)

print("✅ Pydantic Model:")
print(example_policy)
print(f"\n轉換為 JSON:")
print(example_policy.model_dump_json(indent=2))

### 2.2 巢狀結構 (Nested Models)

In [ ]:
class Beneficiary(BaseModel):
    """受益人資訊"""
    name: str = Field(description="受益人姓名")
    relationship: str = Field(description="與投保人關係")
    share_percentage: int = Field(description="受益比例 (1-100)", ge=1, le=100)

class DetailedPolicyInfo(BaseModel):
    """詳細保單資訊 (含受益人)"""
    policy_id: str = Field(description="保單號碼")
    policy_type: str = Field(description="保單類型")
    owner_name: str = Field(description="投保人姓名")
    coverage_amount: int = Field(description="保額", gt=0)
    annual_premium: int = Field(description="年繳保費", gt=0)
    beneficiaries: List[Beneficiary] = Field(description="受益人列表")
    tags: List[str] = Field(default=[], description="保單標籤")

# 測試
detailed_policy = DetailedPolicyInfo(
    policy_id="CL202401001",
    policy_type="終身壽險",
    owner_name="王小明",
    coverage_amount=2000000,
    annual_premium=50000,
    beneficiaries=[
        Beneficiary(name="王太太", relationship="配偶", share_percentage=60),
        Beneficiary(name="王小華", relationship="子女", share_percentage=40)
    ],
    tags=["高保額", "家庭保障"]
)

print("📋 詳細保單資訊:")
print(detailed_policy.model_dump_json(indent=2, ensure_ascii=False))

## Part 3: with_structured_output() 方法

### 3.1 基礎用法

In [ ]:
# 建立結構化輸出的 LLM
structured_llm = llm.with_structured_output(PolicyInfo)

# 呼叫 LLM
result = structured_llm.invoke(
    "請產生一個範例保單: 保單號碼 CL202401001, 終身壽險, 投保人王小明, 保額 200 萬, 年繳 5 萬, 有效, 2024-01-15 生效"
)

print("✅ 結構化輸出:")
print(f"類型: {type(result)}")
print(f"保單號碼: {result.policy_id}")
print(f"投保人: {result.owner_name}")
print(f"保額: ${result.coverage_amount:,}")
print(f"年繳: ${result.annual_premium:,}")
print(f"\nJSON:")
print(result.model_dump_json(indent=2, ensure_ascii=False))

### 3.2 處理複雜查詢

In [ ]:
# 使用巢狀結構
detailed_structured_llm = llm.with_structured_output(DetailedPolicyInfo)

result = detailed_structured_llm.invoke("""
請產生保單資料:
- 保單號碼: CL202401002
- 類型: 醫療險
- 投保人: 李小華
- 保額: 100 萬
- 年繳: 3 萬
- 受益人: 配偶李太太 (70%), 子女李小明 (30%)
- 標籤: 醫療保障, 家庭照顧
""")

print("👨‍👩‍👧 詳細保單資訊:")
print(f"保單號碼: {result.policy_id}")
print(f"投保人: {result.owner_name}")
print(f"\n受益人:")
for b in result.beneficiaries:
    print(f"  - {b.name} ({b.relationship}): {b.share_percentage}%")
print(f"\n標籤: {', '.join(result.tags)}")

## Part 4: Tool Strategy vs Provider Strategy

### 4.1 兩種策略的差異

| 策略 | Tool Strategy | Provider Strategy |
|------|--------------|------------------|
| 實作方式 | 使用 Function Calling | 使用 Provider 原生功能 |
| 支援度 | 所有支援 Function Calling 的模型 | 僅特定 Provider |
| 準確度 | 高 | 非常高 |
| 速度 | 正常 | 較快 |
| 適用場景 | 一般用途 | 需要極高準確度 |

### 4.2 使用 Tool Strategy (預設)

In [ ]:
# 明確指定使用 Tool Strategy
tool_strategy_llm = llm.with_structured_output(
    PolicyInfo,
    method="function_calling"  # Tool Strategy
)

result = tool_strategy_llm.invoke("範例保單: CL999, 壽險, 張三, 300萬, 8萬, active, 2024-02-01")

print("🔧 Tool Strategy 輸出:")
print(result.model_dump_json(indent=2, ensure_ascii=False))

### 4.3 使用 Provider Strategy (JSON Mode)

In [ ]:
# OpenAI 的 JSON Mode
provider_strategy_llm = llm.with_structured_output(
    PolicyInfo,
    method="json_mode"  # Provider Strategy
)

result = provider_strategy_llm.invoke("範例保單: CL888, 醫療, 李四, 150萬, 4萬, active, 2024-03-01")

print("⚡ Provider Strategy 輸出:")
print(result.model_dump_json(indent=2, ensure_ascii=False))

## Part 5: 實戰案例 - 保單資料擷取系統

### 5.1 定義多種保單類型

In [ ]:
from enum import Enum

class PolicyType(str, Enum):
    """保單類型列舉"""
    WHOLE_LIFE = "終身壽險"
    TERM_LIFE = "定期壽險"
    MEDICAL = "醫療險"
    ACCIDENT = "意外險"
    SAVINGS = "儲蓄險"
    ANNUITY = "年金險"

class PolicyStatus(str, Enum):
    """保單狀態"""
    ACTIVE = "有效"
    INACTIVE = "失效"
    PENDING = "等待期"
    CANCELLED = "解約"

class PaymentFrequency(str, Enum):
    """繳費頻率"""
    ANNUAL = "年繳"
    SEMI_ANNUAL = "半年繳"
    QUARTERLY = "季繳"
    MONTHLY = "月繳"

class ComprehensivePolicyInfo(BaseModel):
    """完整保單資訊"""
    policy_id: str = Field(description="保單號碼")
    policy_type: PolicyType = Field(description="保單類型")
    status: PolicyStatus = Field(description="保單狀態")
    owner_name: str = Field(description="投保人姓名")
    insured_name: str = Field(description="被保險人姓名")
    coverage_amount: int = Field(description="保額", gt=0)
    premium: int = Field(description="保費金額", gt=0)
    payment_frequency: PaymentFrequency = Field(description="繳費頻率")
    start_date: str = Field(description="生效日期 YYYY-MM-DD")
    end_date: Optional[str] = Field(None, description="到期日期 YYYY-MM-DD (終身則為 None)")
    beneficiaries: List[Beneficiary] = Field(default=[], description="受益人列表")
    riders: List[str] = Field(default=[], description="附約列表")
    
print("✅ 完整保單結構定義完成")

### 5.2 從非結構化文本擷取資料

In [ ]:
# 建立擷取器
extractor = llm.with_structured_output(ComprehensivePolicyInfo)

# 模擬客服對話紀錄
conversation_log = """
客戶: 我想查詢我的保單資料
客服: 好的,請問您的保單號碼?
客戶: CL202401999
客服: 好的,這是一張終身壽險,投保人和被保險人都是陳大明先生,
      保額 500 萬,月繳保費 12,000 元,2024 年 1 月 1 日生效。
      受益人是配偶陳太太 100%。另外有附加住院醫療和意外傷害附約。
      目前保單狀態是有效的。
"""

# 擷取結構化資料
extracted_policy = extractor.invoke(
    f"從以下對話中擷取保單資訊:\n{conversation_log}"
)

print("📊 擷取結果:")
print("="*60)
print(f"保單號碼: {extracted_policy.policy_id}")
print(f"保單類型: {extracted_policy.policy_type.value}")
print(f"狀態: {extracted_policy.status.value}")
print(f"投保人: {extracted_policy.owner_name}")
print(f"被保險人: {extracted_policy.insured_name}")
print(f"保額: ${extracted_policy.coverage_amount:,}")
print(f"保費: ${extracted_policy.premium:,} ({extracted_policy.payment_frequency.value})")
print(f"生效日期: {extracted_policy.start_date}")
print(f"\n受益人:")
for b in extracted_policy.beneficiaries:
    print(f"  - {b.name} ({b.relationship}): {b.share_percentage}%")
print(f"\n附約: {', '.join(extracted_policy.riders)}")

print(f"\n完整 JSON:")
print(extracted_policy.model_dump_json(indent=2, ensure_ascii=False))

### 5.3 批次處理多筆保單

In [ ]:
class PolicyBatch(BaseModel):
    """多筆保單集合"""
    policies: List[ComprehensivePolicyInfo] = Field(description="保單列表")
    total_count: int = Field(description="保單總數")

batch_extractor = llm.with_structured_output(PolicyBatch)

# 模擬批次資料
batch_data = """
以下是 3 筆保單資料:

1. CL001 - 終身壽險,王小明,300 萬,年繳 6 萬,2024-01-15 生效,有效,受益人配偶 100%
2. CL002 - 醫療險,李小華,150 萬,月繳 3000,2024-02-01 生效,有效,受益人父母各 50%
3. CL003 - 定期壽險,張大同,200 萬,季繳 15000,2024-03-01 生效至 2034-03-01,有效,受益人子女 100%
"""

batch_result = batch_extractor.invoke(f"擷取以下保單資料:\n{batch_data}")

print(f"📦 批次處理結果:")
print(f"總計: {batch_result.total_count} 筆保單\n")

for i, policy in enumerate(batch_result.policies, 1):
    print(f"{i}. {policy.policy_id} - {policy.policy_type.value}")
    print(f"   投保人: {policy.owner_name}")
    print(f"   保額: ${policy.coverage_amount:,}")
    print(f"   保費: ${policy.premium:,} ({policy.payment_frequency.value})")
    print()

## 🎯 練習題

### 練習 1: 理賠資料結構化

定義理賠資料結構,包含:
- 理賠編號、保單號碼、申請人
- 理賠類型 (住院/手術/門診)
- 理賠金額、申請日期、審核狀態
- 需補件列表

In [ ]:
# TODO: 定義 ClaimInfo 結構
# TODO: 從文本擷取理賠資料

### 練習 2: 客戶資料整合

建立客戶完整資料結構,包含:
- 基本資料 (姓名、生日、聯絡方式)
- 擁有的保單列表
- 理賠歷史
- 風險評級

In [ ]:
# TODO: 定義 CustomerProfile 結構

### 練習 3: 保單比較系統

建立保單比較結構,比較 2-3 個保單的:
- 保費差異
- 保障範圍差異
- 推薦結論

In [ ]:
# TODO: 定義 PolicyComparison 結構

## ✅ 練習解答

### 練習 1 解答

In [ ]:
class ClaimType(str, Enum):
    HOSPITALIZATION = "住院"
    SURGERY = "手術"
    OUTPATIENT = "門診"
    DEATH = "身故"

class ClaimStatus(str, Enum):
    PENDING = "審核中"
    APPROVED = "已核准"
    REJECTED = "已拒絕"
    PAID = "已理賠"
    NEED_DOCS = "需補件"

class ClaimInfo(BaseModel):
    """理賠資訊"""
    claim_id: str = Field(description="理賠編號")
    policy_id: str = Field(description="保單號碼")
    applicant_name: str = Field(description="申請人姓名")
    claim_type: ClaimType = Field(description="理賠類型")
    claim_amount: int = Field(description="理賠金額", gt=0)
    application_date: str = Field(description="申請日期 YYYY-MM-DD")
    status: ClaimStatus = Field(description="審核狀態")
    required_documents: List[str] = Field(default=[], description="需補文件")
    notes: Optional[str] = Field(None, description="備註")

# 測試
claim_extractor = llm.with_structured_output(ClaimInfo)

claim_text = """
理賠編號 CLM20240315001,保單 CL202401001,申請人王小明,
因住院申請理賠 5 萬元,2024-03-15 申請,目前需補件,
需提供診斷證明書和醫療費用收據。
"""

claim_result = claim_extractor.invoke(f"擷取理賠資料:\n{claim_text}")

print("🏥 理賠資料:")
print(claim_result.model_dump_json(indent=2, ensure_ascii=False))

## 📝 學習檢查清單

- [ ] 理解結構化輸出的重要性
- [ ] 使用 Pydantic BaseModel 定義資料結構
- [ ] 使用 Field 添加欄位描述和驗證
- [ ] 建立巢狀 Models
- [ ] 使用 Enum 定義列舉類型
- [ ] 使用 with_structured_output() 方法
- [ ] 理解 Tool Strategy vs Provider Strategy
- [ ] 從非結構化文本擷取結構化資料
- [ ] 批次處理多筆資料
- [ ] 整合到實際應用場景

## 下一步
繼續前往 `04-memory-management.ipynb` 學習記憶管理 👉